In [1]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [1]:
from transformers import DebertaConfig, DebertaForMaskedLM

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
from transformers import AutoModel, AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [9]:
config = DebertaConfig(
    hidden_size = 384,
    vocab_size= tokenizer.vocab_size,
    num_hidden_layers = 6,
    num_attention_heads = 6,
    intermediate_size = 1024,
    max_position_embeddings = 256
)

In [12]:
train_file = "/home/mdafifal.mamun/notebooks/triagerX/notebook/data/deeptriage/deep_data.csv"

In [13]:
import pandas as pd

In [14]:
df = pd.read_csv(train_file)

In [15]:
df.head()

,id,issue_id,issue_title,reported_time,owner,description
0,1,2,Testing if chromium id works,2008-08-30 16:00:21,NaN,what steps will reproduce the problem123what i...
1,2,3,This is a test,2008-08-31 02:47:11,NaN,product version 0214927urls if applicabl...
2,3,5,Java not working yet,2008-09-02 19:04:27,NaN,product version 0214927urls if applicabl...
3,4,6,Chrome treats links in frames differently than...,2008-09-02 19:08:01,NaN,product version 0214927 1583urls if appl...
4,5,7,Errors in importing from firefox,2008-09-02 19:08:46,venkataramana@chromium.org,product version 0214927urls if applicabl...


In [16]:
texts = [
    f"Issue Title: {row.issue_title}\t\tIssue Description: {row.description}"
    for _, row in df.iterrows()
]

In [17]:
training_corpus = (
    texts[i : i + 1000]
    for i in range(0, len(texts), 1000)
)

In [12]:
tokenized_texts = tokenizer(texts, truncation=True, padding=True)

In [20]:
tokenizer_trained= tokenizer.train_new_from_iterator(training_corpus, 2_000)

In [21]:
tokens = tokenizer_trained(texts[100]).input_ids
[tokenizer.decode(id) for id in tokens]

['[CLS]',
 ' support',
 ' 15',
 ' by',
 ' website',
 ' price',
 ' form',
 ' Wednesday',
 ' defense',
 ' President',
 ' real',
 'es',
 ' nine',
 ' still',
 ' still',
 ' support',
 ' big',
 ' by',
 'Q',
 ' local',
 ' came',
 ' course',
 ' April',
 ' Thomas',
 ' O',
 ' 2015',
 ' law',
 ' m',
 ' results',
 '.',
 ' members',
 ' whether',
 ' together',
 ' law',
 ' m',
 ' person',
 ' later',
 ' 18',
 ' lead',
 ' months',
 ' meet',
 ' hit',
 ' safety',
 ' hit',
 ' issues',
 ' least',
 ' There',
 ' give',
 ' China',
 ' today',
 'old',
 ' media',
 ' was',
 'io',
 ' limited',
 ' 8',
 ' business',
 ' total',
 '�',
 ' 8',
 ' business',
 ' German',
 ' won',
 ' law',
 ' create',
 'old',
 ' road',
 'se',
 'Reuters',
 ' St',
 ' platform',
 ' German',
 ' won',
 ' trade',
 ' There',
 ' school',
 'old',
 '.,',
 ' Police',
 ' calling',
 'um',
 ' than',
 ' total',
 ' council',
 ' death',
 ' There',
 ' along',
 ' times',
 ' daily',
 ' forces',
 ' run',
 ' China',
 'But',
 ' she',
 ' open',
 'll',
 'am',
 ' 8

In [22]:
tokenizer.save_pretrained("tokenizer/deeptriage")

('tokenizer/deeptriage/tokenizer_config.json',
 'tokenizer/deeptriage/special_tokens_map.json',
 'tokenizer/deeptriage/vocab.json',
 'tokenizer/deeptriage/merges.txt',
 'tokenizer/deeptriage/added_tokens.json',
 'tokenizer/deeptriage/tokenizer.json')

In [52]:
config = DebertaConfig(
    hidden_size = 384,
    vocab_size= tokenizer.vocab_size,
    num_hidden_layers = 6,
    num_attention_heads = 6,
    intermediate_size = 1024,
    max_position_embeddings = 512
)

In [53]:
model = DebertaForMaskedLM(config=config)
print(model.num_parameters()) #10457864

27980121


In [54]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [28]:
df.head()

,id,issue_id,issue_title,reported_time,owner,description
0,1,2,Testing if chromium id works,2008-08-30 16:00:21,NaN,what steps will reproduce the problem123what i...
1,2,3,This is a test,2008-08-31 02:47:11,NaN,product version 0214927urls if applicabl...
2,3,5,Java not working yet,2008-09-02 19:04:27,NaN,product version 0214927urls if applicabl...
3,4,6,Chrome treats links in frames differently than...,2008-09-02 19:08:01,NaN,product version 0214927 1583urls if appl...
4,5,7,Errors in importing from firefox,2008-09-02 19:08:46,venkataramana@chromium.org,product version 0214927urls if applicabl...


In [30]:
df["text"] = [f"Issue Title: {row.issue_title}\t\tIssue Description: {row.description}" for _, row in df.iterrows()]

In [31]:

import torch
from torch.utils.data import Dataset
from accelerate import Accelerator, DistributedType

class LineByLineTextDataset(Dataset):
    def __init__(self, tokenizer, raw_datasets, max_length: int):
        self.padding = "max_length"
        self.text_column_name = 'text'
        self.max_length = max_length
        self.accelerator = Accelerator(gradient_accumulation_steps=1)
        self.tokenizer = tokenizer

        with self.accelerator.main_process_first():
            self.tokenized_datasets = raw_datasets.map(
                self.tokenize_function,
                batched=True,
                num_proc=4,
                remove_columns=[self.text_column_name],
                desc="Running tokenizer on dataset line_by_line",
            )
            self.tokenized_datasets.set_format('torch',columns=['input_ids'],dtype=torch.long)

    def tokenize_function(self,examples):
        examples[self.text_column_name] = [
            line for line in examples[self.text_column_name] if len(line[0]) > 0 and not line[0].isspace()
        ]
        return self.tokenizer(
            examples[self.text_column_name],
            padding=self.padding,
            truncation=True,
            max_length=self.max_length,
            return_special_tokens_mask=True,
        )
    def __len__(self):
        return len(self.tokenized_datasets)

    def __getitem__(self, i):
        return self.tokenized_datasets[i]

In [36]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.3 MB/s eta 0:00:000:00:01m eta 0:00:01
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 34.9 MB/s eta 0:00:000m eta 0:00:010:011
Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)


In [37]:
from datasets import Dataset

In [43]:
from sklearn.model_selection import train_test_split

In [45]:
X_train, y_train = train_test_split(df, test_size=0.2, shuffle=True)

In [47]:
train_dataset = Dataset.from_pandas(X_train)
test_dataset = Dataset.from_pandas(y_train)

In [48]:
train_line_dataset = LineByLineTextDataset(tokenizer_trained, train_dataset, 512)
test_line_dataset = LineByLineTextDataset(tokenizer_trained, test_dataset, 512)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Running tokenizer on dataset line_by_line (num_proc=4):   0%|          | 0/130956 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Running tokenizer on dataset line_by_line (num_proc=4):   0%|          | 0/32739 [00:00<?, ? examples/s]

In [55]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./model",
    overwrite_output_dir=True,
    # push_to_hub=True,
    # hub_model_id="Ransaka/sinhala-bert-yt",
    learning_rate=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    max_steps=1000,
    eval_steps=100,
    logging_steps=100,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    report_to='none',
    # hub_private_repo = True,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_line_dataset,
    eval_dataset= test_line_dataset, # change to your actual evaluation dataset
    )

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [56]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 6.14 GiB. GPU 0 has a total capacity of 15.77 GiB of which 4.43 GiB is free. Including non-PyTorch memory, this process has 11.33 GiB memory in use. Of the allocated memory 10.69 GiB is allocated by PyTorch, and 289.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [14]:
from tqdm import tqdm

In [15]:
with open("mlm_training_data.txt", "w") as f:
    for text in tqdm(texts):
        f.write(text + "\n")

  0%|          | 0/163695 [00:00<?, ?it/s]

100%|██████████| 163695/163695 [00:00<00:00, 508136.63it/s]


In [21]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="mlm_training_data.txt",
    block_size=4096,  # Choose an appropriate block size
)

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [22]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15,  # Probability of masking a token
)

In [23]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
training_args = TrainingArguments(
    output_dir="./roberta-mlm",
    overwrite_output_dir=True,
    num_train_epochs=10,  # Number of pre-training epochs
    per_device_train_batch_size=32,
    save_steps=1000,
    save_total_limit=2,
)

In [25]:
# Initialize Trainer for MLM pre-training
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Start pre-training for MLM
trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


RuntimeError: The expanded size of the tensor (726) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [32, 726].  Tensor sizes: [1, 514]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
